In [ ]:
import re
import spacy
import pandas as pd
from pykospacing import Spacing

from spacy.lang.ko import Korean
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("ko_core_news_sm")

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 불용어 리스트 가져오기
stopwords = get_stopwords()

# 형태소 분석 및 불용어 제거 함수
def analyze_and_clean(text):
    preprocessed_text = preprocessing(text)
    doc = nlp(preprocessed_text)
    tokens = [token.text for token in doc if token.text not in stopwords]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text not in stopwords]   
    return ' '.join(tokens), pos_tags

In [ ]:
df = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')

# 데이터프레임의 'feature' 컬럼에 대해 분석 적용
# df = df[:100]
df['cleaned_feature'], df['pos_tags'] = zip(*df['feature'].apply(analyze_and_clean))

In [ ]:
# 리스트를 문자열로 변환 (불필요할 경우 생략 가능)
df['cleaned_feature_str'] = df['cleaned_feature']

# 비어 있는 문서 또는 공백만 포함된 문서 제거 및 복사본 생성
df_cleaned = df[df['cleaned_feature_str'].apply(lambda x: len(x.strip()) > 0)].copy()

# 비어 있는 문서에 'unknown' 추가 (필요한 경우)
df_cleaned.loc[df_cleaned['cleaned_feature_str'].apply(lambda x: len(x.strip()) == 0), 'cleaned_feature_str'] = 'unknown'

df_cleaned['cleaned_feature_str'] = df_cleaned['cleaned_feature_str'].astype(str)
df_cleaned['cleaned_feature_str'].fillna('', inplace=True)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_cleaned['cleaned_feature_str'])

# 데이터프레임으로 변환
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())